<a href="https://colab.research.google.com/github/jkimbf/FL-Tutorials/blob/master/MNIST_FL_PySyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b ryffel/4P --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets

     |████████████████████████████████| 2.7MB 4.4MB/s 
     |████████████████████████████████| 110.5MB 47kB/s 
     |████████████████████████████████| 276kB 39.3MB/s 
     |████████████████████████████████| 512kB 41.9MB/s 
     |████████████████████████████████| 3.8MB 39.7MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=37840118778b8a89618d836309518e54067ff26d43220263f5287ce38cbbc25b
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ba34658eedc4b2a7d24b8cfb4b4937d053f0bfb8b107f30a4f6dadfa9e870d0e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built pyyaml gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninsta

     |████████████████████████████████| 81kB 3.0MB/s 
ERROR: syft 0.2.4 has requirement lz4~=3.0.2, but you'll have lz4 3.1.0 which is incompatible.
ERROR: syft 0.2.4 has requirement Pillow~=6.2.2, but you'll have pillow 7.0.0 which is incompatible.
ERROR: syft 0.2.4 has requirement requests~=2.22.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: syft 0.2.4 has requirement torch~=1.4.0, but you'll have torch 1.5.0+cu101 which is incompatible.
ERROR: syft 0.2.4 has requirement torchvision~=0.5.0, but you'll have torchvision 0.6.0+cu101 which is incompatible.
  Found existing installation: websockets 8.1
    Uninstalling websockets-8.1:
      Successfully uninstalled websockets-8.1


In [6]:
!pip install torchvision==0.5.0

     |████████████████████████████████| 4.0MB 4.7MB/s 
     |████████████████████████████████| 753.4MB 16kB/s 
ERROR: syft 0.2.4 has requirement lz4~=3.0.2, but you'll have lz4 3.1.0 which is incompatible.
ERROR: syft 0.2.4 has requirement Pillow~=6.2.2, but you'll have pillow 7.0.0 which is incompatible.
ERROR: syft 0.2.4 has requirement requests~=2.22.0, but you'll have requests 2.23.0 which is incompatible.
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
  Found existing installation: torchvision 0.6.0+cu101
    Uninstalling torchvision-0.6.0+cu101:
      Successfully uninstalled torchvision-0.6.0+cu101


In [3]:
!pip install syft_proto

     |████████████████████████████████| 1.3MB 4.1MB/s 
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging

# Importing PySyft to simulate federated learning
import syft as sy

# Hooking PyTorch to PySyft
hook = sy.TorchHook(torch)

# Creating two imaginary clients
school_1 = sy.VirtualWorker(hook, id="westside")
school_2 = sy.VirtualWorker(hook, id="grapevine")

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


## Defining hyper-parameters: learning rate, batch size, test batch size, etc.

In [0]:
# Defining the args
args = {
    'use_cuda' : True,
    'batch_size' : 64,
    'test_batch_size' : 1000,
    'lr' : 0.01,
    'log_interval' : 10,
    'epochs' : 10
}

# Checking to use GPU or not
use_cuda = args['use_cuda'] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Defining a very simple CNN

In [0]:
# creating a simple CNN net
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        self.fc = nn.Sequential(
            nn.Linear(in_features=64*12*12, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10)
        )

        self.dropout = nn.Dropout2d(0.25)

    def forward(self, x):
        x = self.conv(x)
        x = F.max_pool2d(x,2)
        x = x.view(-1, 64*12*12)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

## Sending the data to schools
```.federate()```
* splits the dataset in two parts
* sends this data across two remote workers (two schools)

<br> * No need to send data in real life

In [0]:
# Use PySyft's API to prepare the data and distribute across 2 worker
# This is just to simulate federated learning
federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,),(0.3081,))
                   ]))
    .federate((school_2, school_1)),
    batch_size=args['batch_size'], 
    shuffle=True
)

# test data remains with me locally
# normal torch code to load test data from MNIST
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args['test_batch_size'],
    shuffle=True
)

## Training and Validation functions

> Each time we train the model, need to send it to the right location for each batch (using ```.send()``` function)
<br><br> We perform all the operations **REMOTELY** with the same syntax like we're doing local PyTorch 
<br><br> When we're done, get back the updated model using the ```.get()``` method.

So Simple!

<br>Note that ```(data, target)``` is a pair of ```PointerTensor``` in the ```train``` function below

In a ```PointerTensor```, I can **get the worker** it points to using ```.location``` attribute.

```.location``` attribute is what I need to **use to send the model to the correct location**.

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()

    # Iterating over federated data
    for batch_idx, (data, target) in enumerate(train_loader):

        # Sending the model to the remote location
        model = model.send(data.location)

        # Same torch code
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        # this loss is a ptr to the tensor loss at the remote location
        loss = F.nll_loss(output, target)

        # call backward() on the loss ptr,
        # that will send the command to call backward 
        # on the actual loss tensor present on the remote machine
        loss.backward()

        optimizer.step()

        # Get back the updated model
        model.get()

        if batch_idx % args['log_interval'] == 0:
            # note that the variable loss was also created at remote worker,
            # so I need to explicitly get it back
            loss = loss.get()

            print(
                'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch,
                    batch_idx * args['batch_size'], # no of images done
                    len(train_loader) * args['batch_size'], # total images left
                    100. * batch_idx / len(train_loader),
                    loss.item()
                )
            )


The same test function since it is ran locally on my machine.
<br> (only training happens remotely)

In [0]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # Add losses together
            test_loss += F.nll_loss(output, target, reduction='sum').item()

            # Get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)
        )
    )

## Training!!

In [11]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'])

logging.info("Starting training!!")

for epoch in range(1, args['epochs'] + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.304626
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.184304
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.012875
Train Epoch: 1 [1920/60032 (3%)]	Loss: 1.753484
Train Epoch: 1 [2560/60032 (4%)]	Loss: 1.295024
Train Epoch: 1 [3200/60032 (5%)]	Loss: 0.995949
Train Epoch: 1 [3840/60032 (6%)]	Loss: 0.879802
Train Epoch: 1 [4480/60032 (7%)]	Loss: 0.903575
Train Epoch: 1 [5120/60032 (9%)]	Loss: 0.601575
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.510105
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.471961
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.348593
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.298909
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.332955
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.475853
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.303741
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.319659
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.324476
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.307374
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.518553
Train Epoch: 1 [12800/60032 (